In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import csv
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException, TimeoutException



In [2]:
df = pd.read_csv(r"C:/Users\sinwa/projects/RED_BUS_project/csv_files/Telugana.csv")
df


,Route_name,State_Route_link
0,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
1,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
2,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
5,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
7,Hyderabad to Nirmal,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Hyderabad to Karimnagar,https://www.redbus.in/bus-tickets/hyderabad-to...
9,Kothagudem to Hyderabad,https://www.redbus.in/bus-tickets/kothagudem-t...


In [3]:
driver = webdriver.Chrome()
wait=WebDriverWait(driver, 10)
Bus_names       = []
Bus_types       = []
Start_Time      = []
End_Time        = []
Ratings         = []
Duration        = []
Prices          = []
Seats_Available = []
Route_names     = []
Route_links     = []

for i,r in df.iterrows():
    link=r["State_Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    Element=wait.until(EC.presence_of_element_located((By.XPATH, f"//*[@class='clearfix bus-item  ']")))
    bus_name    = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type    = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time  = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time    = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    duration    = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    ratings      = driver.find_elements(By.CSS_SELECTOR, ".rating-sec.lh-24")
    price       = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats       = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    time.sleep(2)
    # Append data to respective lists
    
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time.append(end_time_elem.text)
    for total_duration_elem in duration:
        Duration.append(total_duration_elem.text)
    for rating in ratings:
        Ratings.append(rating.text)
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)
        
print("Scraping completed")



driver.quit()


Scraping completed


In [ ]:
data = {
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Start_time': Start_Time,
    'End_time': End_Time,
    'Total_duration': Duration,
    'Price': Prices,
    "Seats_Available":Seats_Available,
    "Ratings":Ratings,
    'Route_link': Route_links,
    'Route_name': Route_names
}
data


In [ ]:
import numpy as np

# Find the maximum length
max_len = max(len(Bus_names), len(Bus_types), len(Start_Time), len(End_Time), len(Duration),
              len(Prices), len(Seats_Available), len(Ratings), len(Route_links), len(Route_names))

# Define a helper function to pad lists
def pad_list(lst, length):
    return lst + [None] * (length - len(lst))

# Pad all lists to max length
data = {
    "Bus_name": pad_list(Bus_names, max_len),
    "Bus_type": pad_list(Bus_types, max_len),
    "Start_time": pad_list(Start_Time, max_len),
    "End_time": pad_list(End_Time, max_len),
    "Total_duration": pad_list(Duration, max_len),
    "Price": pad_list(Prices, max_len),
    "Seats_Available": pad_list(Seats_Available, max_len),
    "Ratings": pad_list(Ratings, max_len),
    "Route_link": pad_list(Route_links, max_len),
    "Route_name": pad_list(Route_names, max_len)
}

# Create DataFrame
df = pd.DataFrame(data)


In [33]:

# Save to CSV
df.to_csv('C:/Users/sinwa/projects/RED_BUS_project/Bus_details/TLbuses.csv', index=False)


print("CSV file saved successfully.")

CSV file saved successfully.


In [34]:
df = pd.read_csv(r"C:/Users/sinwa/projects/RED_BUS_project/csv_files/AP.csv")
df

,Route_name,State_Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
2,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
3,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Bangalore to Tirupati,https://www.redbus.in/bus-tickets/bangalore-to...
5,Ongole to Hyderabad,https://www.redbus.in/bus-tickets/ongole-to-hy...
6,Anantapur (andhra pradesh) to Bangalore,https://www.redbus.in/bus-tickets/ananthapur-t...
7,Bangalore to Kadapa,https://www.redbus.in/bus-tickets/bangalore-to...
8,Chittoor (Andhra Pradesh) to Bangalore,https://www.redbus.in/bus-tickets/chittoor-and...
9,Bangalore to Anantapur (andhra pradesh),https://www.redbus.in/bus-tickets/bangalore-to...


In [37]:
driver = webdriver.Chrome()
wait=WebDriverWait(driver, 10)
Bus_names       = []
Bus_types       = []
Start_Time      = []
End_Time        = []
Ratings         = []
Duration        = []
Prices          = []
Seats_Available = []
Route_names     = []
Route_links     = []

for i,r in df.iterrows():
    link=r["State_Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    Element=wait.until(EC.presence_of_element_located((By.XPATH, f"//*[@class='clearfix bus-item  ']")))
    bus_name    = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type    = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time  = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time    = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    duration    = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    ratings      = driver.find_elements(By.CSS_SELECTOR, ".rating-sec.lh-24")
    price       = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats       = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    time.sleep(2)
    # Append data to respective lists
    
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time.append(end_time_elem.text)
    for total_duration_elem in duration:
        Duration.append(total_duration_elem.text)
    for rating in ratings:
        Ratings.append(rating.text)
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)
        
print("Scraping completed")



driver.quit()


Scraping completed


In [ ]:
data = {
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Start_time': Start_Time,
    'End_time': End_Time,
    'Total_duration': Duration,
    'Price': Prices,
    "Seats_Available":Seats_Available,
    "Ratings":Ratings,
    'Route_link': Route_links,
    'Route_name': Route_names
}
data

In [39]:
# Find the maximum length
max_len = max(len(Bus_names), len(Bus_types), len(Start_Time), len(End_Time), len(Duration),
              len(Prices), len(Seats_Available), len(Ratings), len(Route_links), len(Route_names))

# Define a helper function to pad lists
def pad_list(lst, length):
    return lst + [None] * (length - len(lst))

# Pad all lists to max length
data = {
    "Bus_name": pad_list(Bus_names, max_len),
    "Bus_type": pad_list(Bus_types, max_len),
    "Start_time": pad_list(Start_Time, max_len),
    "End_time": pad_list(End_Time, max_len),
    "Total_duration": pad_list(Duration, max_len),
    "Price": pad_list(Prices, max_len),
    "Seats_Available": pad_list(Seats_Available, max_len),
    "Ratings": pad_list(Ratings, max_len),
    "Route_link": pad_list(Route_links, max_len),
    "Route_name": pad_list(Route_names, max_len)
}

# Create DataFrame
df = pd.DataFrame(data)

In [40]:
# Save to CSV
df.to_csv('C:/Users/sinwa/projects/RED_BUS_project/Bus_details/AP_Buses.csv', index=False)


print("CSV file saved successfully.")

CSV file saved successfully.


In [8]:
df =pd.read_csv(r"C:/Users/sinwa/projects/RED_BUS_project/csv_files/Jammu.csv")
df

,Route_name,State_Route_link
0,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...
1,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...
2,Srinagar to Jammu (j and k),https://www.redbus.in/bus-tickets/srinagar-to-...
3,Delhi to Jammu (j and k),https://www.redbus.in/bus-tickets/delhi-to-jammu
4,Jammu (j and k) to Delhi,https://www.redbus.in/bus-tickets/jammu-to-delhi
5,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...
6,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...
7,Amritsar to Jammu (j and k),https://www.redbus.in/bus-tickets/amritsar-to-...
8,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...
9,Jaipur (Rajasthan) to Delhi,https://www.redbus.in/bus-tickets/jaipur-to-delhi


In [9]:
driver = webdriver.Chrome()
wait=WebDriverWait(driver, 20)
Bus_names       = []
Bus_types       = []
Start_Time      = []
End_Time        = []
Ratings         = []
Duration        = []
Prices          = []
Seats_Available = []
Route_names     = []
Route_links     = []

for i,r in df.iterrows():
    link=r["State_Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    Element=wait.until(EC.presence_of_element_located((By.XPATH, f"//*[@class='clearfix bus-item  ']")))
    bus_name    = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type    = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time  = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time    = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    duration    = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    ratings      = driver.find_elements(By.CSS_SELECTOR, ".rating-sec.lh-24")
    price       = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats       = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    time.sleep(2)
    # Append data to respective lists
    
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time.append(end_time_elem.text)
    for total_duration_elem in duration:
        Duration.append(total_duration_elem.text)
    for rating in ratings:
        Ratings.append(rating.text)
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)
        
print("Scraping completed")



driver.quit()


Scraping completed


In [10]:
data = {
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Start_time': Start_Time,
    'End_time': End_Time,
    'Total_duration': Duration,
    'Price': Prices,
    "Seats_Available":Seats_Available,
    "Ratings":Ratings,
    'Route_link': Route_links,
    'Route_name': Route_names
}
data

{'Bus_name': ['IntrCity SmartBus',
  'Laxmi holidays',
  'Ram Dalal Holidays Pvt Ltd',
  'AdGULSHAN BUS',
  'Ram Dalal Holidays Pvt Ltd',
  'Deltin Travels Pvt Ltd',
  'Ashiana Travels(India)',
  'AdShub Yatra Holidays',
  'RK Holiday',
  'Awadh Travels',
  'RSRTC - 202391',
  'RSRTC - 191024',
  'RSRTC - 200115',
  'RSRTC - 157466',
  'RSRTC - 201573',
  'RSRTC - 159996',
  'RSRTC - 192474',
  'RSRTC - 169078',
  'RSRTC - 195457',
  'RSRTC - 195619',
  'Goldline Super Deluxe',
  'Goldline Super Deluxe',
  'Goldline Super Deluxe',
  'AdGoldline Super Deluxe',
  'Zingbus Plus'],
 'Bus_type': ['A/C Seater / Sleeper (2+1)',
  'Bharat Benz A/C Seater /Sleeper (2+1)',
  'Bharat Benz A/C Sleeper (2+1)',
  'A/C Seater / Sleeper (2+2)',
  'Bharat Benz A/C Sleeper (2+1)',
  'Bharat Benz A/C Seater /Sleeper (2+1)',
  'A/C Seater / Sleeper (2+2)',
  'A/C Sleeper (2+2)',
  'A/C Seater / Sleeper (2+2)',
  'A/C Sleeper (2+2)',
  'Deluxe AC Seater 2+2',
  'NON AC Seater / Sleeper 2+1',
  'A/C Seater 

In [11]:
# Find the maximum length
max_len = max(len(Bus_names), len(Bus_types), len(Start_Time), len(End_Time), len(Duration),
              len(Prices), len(Seats_Available), len(Ratings), len(Route_links), len(Route_names))

# Define a helper function to pad lists
def pad_list(lst, length):
    return lst + [None] * (length - len(lst))

# Pad all lists to max length
data = {
    "Bus_name": pad_list(Bus_names, max_len),
    "Bus_type": pad_list(Bus_types, max_len),
    "Start_time": pad_list(Start_Time, max_len),
    "End_time": pad_list(End_Time, max_len),
    "Total_duration": pad_list(Duration, max_len),
    "Price": pad_list(Prices, max_len),
    "Seats_Available": pad_list(Seats_Available, max_len),
    "Ratings": pad_list(Ratings, max_len),
    "Route_link": pad_list(Route_links, max_len),
    "Route_name": pad_list(Route_names, max_len)
}

# Create DataFrame
df = pd.DataFrame(data)

In [12]:
# Save to CSV
df.to_csv('C:/Users/sinwa/projects/RED_BUS_project/Bus_details/JK_Buses.csv', index=False)



print("CSV file saved successfully.")

CSV file saved successfully.


In [13]:
df =pd.read_csv(r"C:/Users/sinwa/projects/RED_BUS_project/csv_files/Goa.csv")
df


,Route_name,State_Route_link
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
1,Bangalore to Goa,https://www.redbus.in/bus-tickets/bangalore-to...
2,Mumbai to Goa,https://www.redbus.in/bus-tickets/mumbai-to-goa
3,Goa to Pune,https://www.redbus.in/bus-tickets/goa-to-pune
4,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa
5,Goa to Mumbai,https://www.redbus.in/bus-tickets/goa-to-mumbai
6,Goa to Belagavi,https://www.redbus.in/bus-tickets/goa-to-belagavi
7,Goa to Bangalore,https://www.redbus.in/bus-tickets/goa-to-banga...
8,Pandharpur to Goa,https://www.redbus.in/bus-tickets/pandharpur-t...
9,Goa to Pandharpur,https://www.redbus.in/bus-tickets/goa-to-pandh...


In [26]:
driver = webdriver.Chrome()
driver.maximize_window()
wait=WebDriverWait(driver,10)
Bus_names       = []
Bus_types       = []
Start_Time      = []
End_Time        = []
Ratings         = []
Duration        = []
Prices          = []
Seats_Available = []
Route_names     = []
Route_links     = []

for i,r in df.iterrows():
    link=r["State_Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
        time.sleep(2)
    except:
        continue  
    time.sleep(2)
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    Element=wait.until(EC.presence_of_element_located((By.XPATH, f"//*[@class='clearfix bus-item  ']")))
    bus_name    = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type    = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time  = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time    = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    duration    = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    ratings      = driver.find_elements(By.CSS_SELECTOR, ".rating-sec.lh-24")
    price       = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats       = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    time.sleep(2)
    # Append data to respective lists
    
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time.append(end_time_elem.text)
    for total_duration_elem in duration:
        Duration.append(total_duration_elem.text)
    for rating in ratings:
        Ratings.append(rating.text)
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)
        
print("Scraping completed")



driver.quit()

Scraping completed


In [27]:
data = {
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Start_time': Start_Time,
    'End_time': End_Time,
    'Total_duration': Duration,
    'Price': Prices,
    "Seats_Available":Seats_Available,
    "Ratings":Ratings,
    'Route_link': Route_links,
    'Route_name': Route_names
}
data

{'Bus_name': ['Kadamba Transport Corporation Limited (KTCL) - 89821',
  'Kadamba Transport Corporation Limited (KTCL) - 180183',
  'Atmaram Gobus',
  'Atmaram Gobus',
  'Ashray Travels Benz',
  'AdAnand Tours And Travels',
  'Atmaram Gobus',
  'Zingbus Plus',
  'Ashray Travels Club',
  'AdPrasanna - Purple Bus',
  'Zingbus Plus',
  'Laxmi holidays',
  'Kadamba Transport Corporation Limited (KTCL) - 97386',
  'Kadamba Transport Corporation Limited (KTCL) - 182011',
  'Laxmi holidays',
  'IntrCity SmartBus',
  'Zingbus Plus',
  'AdLaxmi holidays',
  'Dolphin Travel House (37)',
  'IntrCity SmartBus',
  'IntrCity SmartBus',
  'AdAtmaram Travels',
  'Zingbus Plus',
  'Zingbus Plus',
  'Kadamba Transport Corporation Limited (KTCL) - 180182',
  'Kadamba Transport Corporation Limited (KTCL) - 89820',
  'Atmaram Gobus',
  'Atmaram Gobus',
  'Atmaram Gobus',
  'AdAtmaram Travels (Humsafar)',
  'Ashray Travels',
  'Ashray Travels Club',
  'IntrCity SmartBus',
  'AdPrasanna - Purple Bus',
  'Anan

In [ ]:
# Find the maximum length
max_len = max(len(Bus_names), len(Bus_types), len(Start_Time), len(End_Time), len(Duration),
              len(Prices), len(Seats_Available), len(Ratings), len(Route_links), len(Route_names))

# Define a helper function to pad lists
def pad_list(lst, length):
    return lst + [None] * (length - len(lst))

# Pad all lists to max length
data = {
    "Bus_name": pad_list(Bus_names, max_len),
    "Bus_type": pad_list(Bus_types, max_len),
    "Start_time": pad_list(Start_Time, max_len),
    "End_time": pad_list(End_Time, max_len),
    "Total_duration": pad_list(Duration, max_len),
    "Price": pad_list(Prices, max_len),
    "Seats_Available": pad_list(Seats_Available, max_len),
    "Ratings": pad_list(Ratings, max_len),
    "Route_link": pad_list(Route_links, max_len),
    "Route_name": pad_list(Route_names, max_len)
}

# Create DataFrame
df = pd.DataFrame(data)

In [29]:
# Save to CSV
df.to_csv('C:/Users/sinwa/projects/RED_BUS_project/Bus_details/Goa_Buses.csv', index=False)



print("CSV file saved successfully.")

CSV file saved successfully.


In [33]:
df =pd.read_csv(r"C:\Users\sinwa\projects\RED_BUS_project\csv_files\Kerala.csv")
df 


,Route_name,State_Route_link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
5,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
6,Kozhikode to Thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...
7,Thrissur to Kozhikode,https://www.redbus.in/bus-tickets/thrissur-to-...
8,Bangalore to Kannur (Kerala),https://www.redbus.in/bus-tickets/bangalore-to...


In [ ]:
driver = webdriver.Chrome()
driver.maximize_window()
wait=WebDriverWait(driver, 10)
Bus_names       = []
Bus_types       = []
Start_Time      = []
End_Time        = []
Ratings         = []
Duration        = []
Prices          = []
Seats_Available = []
Route_names     = []
Route_links     = []

for i,r in df.iterrows():
    link=r["State_Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver.get(link)
    time.sleep(5)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
        time.sleep(2)
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    Element=wait.until(EC.presence_of_element_located((By.XPATH, f"//*[@class='clearfix bus-item  ']")))
    bus_name    = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type    = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time  = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time    = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    duration    = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    ratings      = driver.find_elements(By.CSS_SELECTOR, ".rating-sec.lh-24")
    price       = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats       = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    time.sleep(2)
    # Append data to respective lists
    
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time.append(end_time_elem.text)
    for total_duration_elem in duration:
        Duration.append(total_duration_elem.text)
    for rating in ratings:
        Ratings.append(rating.text)
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)
        
print("Scraping completed")



driver.quit()


Scraping completed


In [39]:
data= {
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Start_time': Start_Time,
    'End_time': End_Time,
    'Total_duration': Duration,
    'Price': Prices,
    "Seats_Available":Seats_Available,
    "Ratings":Ratings,
    'Route_link': Route_links,
    'Route_name': Route_names
}
data

{'Bus_name': ['KSRTC (Kerala) - 1585',
  'KSRTC (Kerala) - 1586',
  'KSRTC (Kerala) - 2133',
  'KSRTC (Kerala) - 54',
  'KSRTC (Kerala) - 574',
  'KSRTC (Kerala) - 583',
  'KSRTC (Kerala) - 56',
  'ESTEEM TOURS & TRAVELS',
  'MMK Travels',
  'SKS Tours & Travels',
  'AdSuraj Holidays',
  'DLT BUS',
  'SAAM BUS',
  'NS Transports',
  'Highline Transports',
  'NS Transports',
  'National travels and logistics',
  'KSRTC (Kerala) - 2393',
  'KSRTC (Kerala) - 3090',
  'KSRTC (Kerala) - 827',
  'KSRTC (Kerala) - 531',
  'KSRTC (Kerala) - 2454',
  'KSRTC (Kerala) - 1704',
  'KSRTC (Kerala) - 582',
  'KSRTC (Kerala) - 4155',
  'A1 Travels',
  'A1 Travels',
  'Galaxy Tours',
  'A1 Travels',
  'A1 Travels',
  'CITI TOURS',
  'Sonia Travels',
  'Madhavi Travels',
  'UFO Bus',
  'UFO Bus',
  'KSRTC (Kerala) - 1583',
  'KSRTC (Kerala) - 234',
  'KSRTC (Kerala) - 1584',
  'KSRTC (Kerala) - 50',
  'ESTEEM TOURS & TRAVELS',
  'DLT BUS',
  'MMK Travels',
  'SKS Tours & Travels',
  'Highline Transports

In [40]:
# Find the maximum length
max_len = max(len(Bus_names), len(Bus_types), len(Start_Time), len(End_Time), len(Duration),
              len(Prices), len(Seats_Available), len(Ratings), len(Route_links), len(Route_names))

# Define a helper function to pad lists
def pad_list(lst, length):
    return lst + [None] * (length - len(lst))

# Pad all lists to max length
data = {
    "Bus_name": pad_list(Bus_names, max_len),
    "Bus_type": pad_list(Bus_types, max_len),
    "Start_time": pad_list(Start_Time, max_len),
    "End_time": pad_list(End_Time, max_len),
    "Total_duration": pad_list(Duration, max_len),
    "Price": pad_list(Prices, max_len),
    "Seats_Available": pad_list(Seats_Available, max_len),
    "Ratings": pad_list(Ratings, max_len),
    "Route_link": pad_list(Route_links, max_len),
    "Route_name": pad_list(Route_names, max_len)
}

# Create DataFrame
df = pd.DataFrame(data)

In [41]:
# Save to CSV
df.to_csv('C:/Users/sinwa/projects/RED_BUS_project/Bus_details/Kerala_Buses.csv', index=False)



print("CSV file saved successfully.")

CSV file saved successfully.


In [9]:
df =pd.read_csv(r"C:\Users\sinwa\projects\RED_BUS_project\csv_files\Assam.csv")
df

,Route_name,State_Route_link
0,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
1,Guwahati to Nagaon (Assam),https://www.redbus.in/bus-tickets/guwahati-to-...
2,Jorhat to North Lakhimpur,https://www.redbus.in/bus-tickets/jorhat-to-no...
3,Goalpara to Guwahati,https://www.redbus.in/bus-tickets/goalpara-to-...
4,North Lakhimpur to Sibsagar (Assam),https://www.redbus.in/bus-tickets/north-lakhim...
5,North Lakhimpur to Jorhat,https://www.redbus.in/bus-tickets/north-lakhim...
6,Sibsagar (Assam) to North Lakhimpur,https://www.redbus.in/bus-tickets/sibsagar-to-...
7,Dhekiajuli to Guwahati,https://www.redbus.in/bus-tickets/dhekiajuli-t...
8,Jorhat to Dibrugarh,https://www.redbus.in/bus-tickets/jorhat-to-di...
9,Guwahati to Dhubri,https://www.redbus.in/bus-tickets/guwahati-to-...


In [19]:
driver = webdriver.Chrome()
driver.maximize_window()
wait=WebDriverWait(driver, 10)
Bus_names       = []
Bus_types       = []
Start_Time      = []
End_Time        = []
Ratings         = []
Duration        = []
Prices          = []
Seats_Available = []
Route_names     = []
Route_links     = []

for i,r in df.iterrows():
    link=r["State_Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver.get(link)
    time.sleep(3)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    Element=wait.until(EC.presence_of_element_located((By.XPATH, f"//*[@class='clearfix bus-item  ']")))
    bus_name    = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type    = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time  = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time    = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    duration    = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    ratings      = driver.find_elements(By.CSS_SELECTOR, ".rating-sec.lh-24")
    price       = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats       = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    # Append data to respective lists
    
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time.append(end_time_elem.text)
    for total_duration_elem in duration:
        Duration.append(total_duration_elem.text)
    for rating in ratings:
        Ratings.append(rating.text)
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)
        
print("Scraping completed")



driver.quit()


Scraping completed


In [20]:
data= {
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Start_time': Start_Time,
    'End_time': End_Time,
    'Total_duration': Duration,
    'Price': Prices,
    "Seats_Available":Seats_Available,
    "Ratings":Ratings,
    'Route_link': Route_links,
    'Route_name': Route_names
}
data

{'Bus_name': ['Assam State Transport Corporation (ASTC) - 154910',
  'Assam State Transport Corporation (ASTC) - 79324',
  'Mahi Travels(Under ASTC)',
  'Mahi Travels(Under ASTC)',
  'Upasana Travels (Under ASTC)',
  'ANIRUDDHA TRAVELS',
  'Orient Transline',
  'Orient Transline',
  'Orient Transline',
  'Mahi Travels(Under ASTC)',
  'Mahalaxmi Travels',
  'Yatra Travels',
  'WARISPIYA TRAVELS',
  'Kalpana Travels',
  'WARISPIYA TRAVELS',
  'Karisma Aarna',
  'WARISPIYA TRAVELS'],
 'Bus_type': ['Bharat Benz A/C Seater (2+2)',
  'Volvo A/C Seater (2+2)',
  'Bharat Benz A/C Seater (2+2)',
  'A/C Seater (2+2)',
  'Non AC Seater (2+2)',
  'A/C Seater Push Back (2+2)',
  'A/C Seater (2+1)',
  'A/C Seater (2+1)',
  'A/C Seater (2+1)',
  'A/C Seater (2+2)',
  'A/C Seater (2+2)',
  'NON AC Seater/ Sleeper (2+1)',
  'AC Seater (2+2)',
  'AC Seater (2+2)',
  'AC Seater (2+2)',
  'A/C Seater Push Back (2+1)',
  'Bharat Benz A/C Seater (2+2)'],
 'Start_time': ['10:30',
  '13:45',
  '10:30',
  '10:

In [21]:
# Find the maximum length
max_len = max(len(Bus_names), len(Bus_types), len(Start_Time), len(End_Time), len(Duration),
              len(Prices), len(Seats_Available), len(Ratings), len(Route_links), len(Route_names))

# Define a helper function to pad lists
def pad_list(lst, length):
    return lst + [None] * (length - len(lst))

# Pad all lists to max length
data = {
    "Bus_name": pad_list(Bus_names, max_len),
    "Bus_type": pad_list(Bus_types, max_len),
    "Start_time": pad_list(Start_Time, max_len),
    "End_time": pad_list(End_Time, max_len),
    "Total_duration": pad_list(Duration, max_len),
    "Price": pad_list(Prices, max_len),
    "Seats_Available": pad_list(Seats_Available, max_len),
    "Ratings": pad_list(Ratings, max_len),
    "Route_link": pad_list(Route_links, max_len),
    "Route_name": pad_list(Route_names, max_len)
}

# Create DataFrame
df = pd.DataFrame(data)

In [22]:
# Save to CSV
df.to_csv('C:/Users/sinwa/projects/RED_BUS_project/Bus_details/Assam_Buses.csv', index=False)



print("CSV file saved successfully.")

CSV file saved successfully.


In [4]:
df=pd.read_csv(r"C:\Users\sinwa\projects\RED_BUS_project\csv_files\Chandigarh.csv")
df

,Route_name,State_Route_link
0,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
1,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...
2,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...
3,Ludhiana to Chandigarh,https://www.redbus.in/bus-tickets/ludhiana-to-...
4,Chandigarh to Vrindavan,https://www.redbus.in/bus-tickets/chandigarh-t...
5,Chandigarh to Yamuna Nagar,https://www.redbus.in/bus-tickets/chandigarh-t...
6,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
7,Vrindavan to Chandigarh,https://www.redbus.in/bus-tickets/vrindavan-to...
8,Chandigarh to Dehradun,https://www.redbus.in/bus-tickets/chandigarh-t...
9,Dehradun to Chandigarh,https://www.redbus.in/bus-tickets/dehradun-to-...


In [5]:
driver = webdriver.Chrome()
wait=WebDriverWait(driver, 10)
Bus_names       = []
Bus_types       = []
Start_Time      = []
End_Time        = []
Ratings         = []
Duration        = []
Prices          = []
Seats_Available = []
Route_names     = []
Route_links     = []

for i,r in df.iterrows():
    link=r["State_Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    Element=wait.until(EC.presence_of_element_located((By.XPATH, f"//*[@class='clearfix bus-item  ']")))
    bus_name    = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type    = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time  = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time    = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    duration    = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    ratings      = driver.find_elements(By.CSS_SELECTOR, ".rating-sec.lh-24")
    price       = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats       = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    # Append data to respective lists
    
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time.append(end_time_elem.text)
    for total_duration_elem in duration:
        Duration.append(total_duration_elem.text)
    for rating in ratings:
        Ratings.append(rating.text)
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)
        
print("Scraping completed")



driver.quit()


Scraping completed


In [ ]:
data= {
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Start_time': Start_Time,
    'End_time': End_Time,
    'Total_duration': Duration,
    'Price': Prices,
    "Seats_Available":Seats_Available,
    "Ratings":Ratings,
    'Route_link': Route_links,
    'Route_name': Route_names
}
data

In [7]:
# Find the maximum length
max_len = max(len(Bus_names), len(Bus_types), len(Start_Time), len(End_Time), len(Duration),
              len(Prices), len(Seats_Available), len(Ratings), len(Route_links), len(Route_names))

# Define a helper function to pad lists
def pad_list(lst, length):
    return lst + [None] * (length - len(lst))

# Pad all lists to max length
data = {
    "Bus_name": pad_list(Bus_names, max_len),
    "Bus_type": pad_list(Bus_types, max_len),
    "Start_time": pad_list(Start_Time, max_len),
    "End_time": pad_list(End_Time, max_len),
    "Total_duration": pad_list(Duration, max_len),
    "Price": pad_list(Prices, max_len),
    "Seats_Available": pad_list(Seats_Available, max_len),
    "Ratings": pad_list(Ratings, max_len),
    "Route_link": pad_list(Route_links, max_len),
    "Route_name": pad_list(Route_names, max_len)
}

# Create DataFrame
df = pd.DataFrame(data)

In [8]:
# Save to CSV
df.to_csv('C:/Users/sinwa/projects/RED_BUS_project/Bus_details/Chandigarh_Buses.csv', index=False)



print("CSV file saved successfully.")

CSV file saved successfully.


In [23]:
df=pd.read_csv(r"C:/Users/sinwa/projects/RED_BUS_project/csv_files/UP.csv")
df


,Route_name,State_Route_link
0,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...
1,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
2,Lucknow to Delhi,https://www.redbus.in/bus-tickets/lucknow-to-d...
3,Aligarh (uttar pradesh) to Delhi,https://www.redbus.in/bus-tickets/aligarh-utta...
4,Farrukhabad (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/farrukhabad-...
5,Lucknow to Prayagraj(Uttar Pradesh),https://www.redbus.in/bus-tickets/lucknow-to-a...
6,Delhi to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/delhi-to-ali...
7,Lucknow to Agra,https://www.redbus.in/bus-tickets/lucknow-to-agra
8,Delhi to Lucknow,https://www.redbus.in/bus-tickets/delhi-to-luc...
9,Delhi to Moradabad,https://www.redbus.in/bus-tickets/delhi-to-mor...


In [24]:
driver = webdriver.Chrome()
driver.maximize_window()
wait=WebDriverWait(driver, 10)
Bus_names       = []
Bus_types       = []
Start_Time      = []
End_Time        = []
Ratings         = []
Duration        = []
Prices          = []
Seats_Available = []
Route_names     = []
Route_links     = []

for i,r in df.iterrows():
    link=r["State_Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    Element=wait.until(EC.presence_of_element_located((By.XPATH, f"//*[@class='clearfix bus-item  ']")))
    bus_name    = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type    = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time  = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time    = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    duration    = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    ratings      = driver.find_elements(By.CSS_SELECTOR, ".rating-sec.lh-24")
    price       = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats       = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    # Append data to respective lists
    
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time.append(end_time_elem.text)
    for total_duration_elem in duration:
        Duration.append(total_duration_elem.text)
    for rating in ratings:
        Ratings.append(rating.text)
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)
        
print("Scraping completed")



driver.quit()


Scraping completed


In [ ]:
data= {
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Start_time': Start_Time,
    'End_time': End_Time,
    'Total_duration': Duration,
    'Price': Prices,
    "Seats_Available":Seats_Available,
    "Ratings":Ratings,
    'Route_link': Route_links,
    'Route_name': Route_names
}
data

In [26]:
# Find the maximum length
max_len = max(len(Bus_names), len(Bus_types), len(Start_Time), len(End_Time), len(Duration),
              len(Prices), len(Seats_Available), len(Ratings), len(Route_links), len(Route_names))

# Define a helper function to pad lists
def pad_list(lst, length):
    return lst + [None] * (length - len(lst))

# Pad all lists to max length
data = {
    "Bus_name": pad_list(Bus_names, max_len),
    "Bus_type": pad_list(Bus_types, max_len),
    "Start_time": pad_list(Start_Time, max_len),
    "End_time": pad_list(End_Time, max_len),
    "Total_duration": pad_list(Duration, max_len),
    "Price": pad_list(Prices, max_len),
    "Seats_Available": pad_list(Seats_Available, max_len),
    "Ratings": pad_list(Ratings, max_len),
    "Route_link": pad_list(Route_links, max_len),
    "Route_name": pad_list(Route_names, max_len)
}

# Create DataFrame
df = pd.DataFrame(data)

In [27]:
# Save to CSV
df.to_csv('C:/Users/sinwa/projects/RED_BUS_project/Bus_details/UP_Buses.csv', index=False)



print("CSV file saved successfully.")

CSV file saved successfully.


In [28]:
df=pd.read_csv(r"C:/Users/sinwa/projects/RED_BUS_project/csv_files/Rajasthan.csv")
df

,Route_name,State_Route_link
0,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
1,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
2,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
3,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
4,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
5,Jaipur (Rajasthan) to Pilani,https://www.redbus.in/bus-tickets/jaipur-to-pi...
6,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
7,Kishangarh (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...
8,Jodhpur to Beawar (Rajasthan),https://www.redbus.in/bus-tickets/jodhpur-to-b...


In [30]:
driver = webdriver.Chrome()
driver.maximize_window()
wait=WebDriverWait(driver, 10)
Bus_names       = []
Bus_types       = []
Start_Time      = []
End_Time        = []
Ratings         = []
Duration        = []
Prices          = []
Seats_Available = []
Route_names     = []
Route_links     = []

for i,r in df.iterrows():
    link=r["State_Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(2)  
        
        new_page_source = driver.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    Element=wait.until(EC.presence_of_element_located((By.XPATH, f"//*[@class='clearfix bus-item  ']")))
    bus_name    = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type    = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time  = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time    = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    duration    = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    ratings      = driver.find_elements(By.CSS_SELECTOR, ".rating-sec.lh-24")
    price       = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats       = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    # Append data to respective lists
    
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time.append(end_time_elem.text)
    for total_duration_elem in duration:
        Duration.append(total_duration_elem.text)
    for rating in ratings:
        Ratings.append(rating.text)
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)
        
print("Scraping completed")



driver.quit()


Scraping completed


In [ ]:
data= {
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Start_time': Start_Time,
    'End_time': End_Time,
    'Total_duration': Duration,
    'Price': Prices,
    "Seats_Available":Seats_Available,
    "Ratings":Ratings,
    'Route_link': Route_links,
    'Route_name': Route_names
}
data

In [32]:
# Find the maximum length
max_len = max(len(Bus_names), len(Bus_types), len(Start_Time), len(End_Time), len(Duration),
              len(Prices), len(Seats_Available), len(Ratings), len(Route_links), len(Route_names))

# Define a helper function to pad lists
def pad_list(lst, length):
    return lst + [None] * (length - len(lst))

# Pad all lists to max length
data = {
    "Bus_name": pad_list(Bus_names, max_len),
    "Bus_type": pad_list(Bus_types, max_len),
    "Start_time": pad_list(Start_Time, max_len),
    "End_time": pad_list(End_Time, max_len),
    "Total_duration": pad_list(Duration, max_len),
    "Price": pad_list(Prices, max_len),
    "Seats_Available": pad_list(Seats_Available, max_len),
    "Ratings": pad_list(Ratings, max_len),
    "Route_link": pad_list(Route_links, max_len),
    "Route_name": pad_list(Route_names, max_len)
}

# Create DataFrame
df = pd.DataFrame(data)

In [33]:
# Save to CSV
df.to_csv('C:/Users/sinwa/projects/RED_BUS_project/Bus_details/Rajasthan_Buses.csv', index=False)



print("CSV file saved successfully.")

CSV file saved successfully.


In [34]:
df =pd.read_csv(r"C:/Users/sinwa/projects/RED_BUS_project/csv_files/Himachal.csv")
df

,Route_name,State_Route_link
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
2,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
3,Shimla to Delhi,https://www.redbus.in/bus-tickets/shimla-to-delhi
4,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
5,Chamba (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/chamba-himac...
6,Shimla to Chandigarh,https://www.redbus.in/bus-tickets/shimla-to-ch...
7,Dharamshala (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/dharamshala-...
8,Delhi to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-ham...
9,Shimla to Manali,https://www.redbus.in/bus-tickets/shimla-to-ma...


In [37]:
driver = webdriver.Chrome()
driver.maximize_window()
wait=WebDriverWait(driver, 10)
Bus_names       = []
Bus_types       = []
Start_Time      = []
End_Time        = []
Ratings         = []
Duration        = []
Prices          = []
Seats_Available = []
Route_names     = []
Route_links     = []

for i,r in df.iterrows():
    link=r["State_Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(4)
        
    # click elements to views bus
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(2)  
        
        new_page_source = driver.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    Element=wait.until(EC.presence_of_element_located((By.XPATH, f"//*[@class='clearfix bus-item  ']")))
    bus_name    = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type    = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time  = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time    = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    duration    = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    ratings      = driver.find_elements(By.CSS_SELECTOR, ".rating-sec.lh-24")
    price       = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats       = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    
    # Append data to respective lists
    
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time.append(end_time_elem.text)
    for total_duration_elem in duration:
        Duration.append(total_duration_elem.text)
    for rating in ratings:
        Ratings.append(rating.text)
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)
        
print("Scraping completed")



driver.quit()

Scraping completed


In [ ]:
data= {
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Start_time': Start_Time,
    'End_time': End_Time,
    'Total_duration': Duration,
    'Price': Prices,
    "Seats_Available":Seats_Available,
    "Ratings":Ratings,
    'Route_link': Route_links,
    'Route_name': Route_names
}
data

In [39]:
# Find the maximum length
max_len = max(len(Bus_names), len(Bus_types), len(Start_Time), len(End_Time), len(Duration),
              len(Prices), len(Seats_Available), len(Ratings), len(Route_links), len(Route_names))

# Define a helper function to pad lists
def pad_list(lst, length):
    return lst + [None] * (length - len(lst))

# Pad all lists to max length
data = {
    "Bus_name": pad_list(Bus_names, max_len),
    "Bus_type": pad_list(Bus_types, max_len),
    "Start_time": pad_list(Start_Time, max_len),
    "End_time": pad_list(End_Time, max_len),
    "Total_duration": pad_list(Duration, max_len),
    "Price": pad_list(Prices, max_len),
    "Seats_Available": pad_list(Seats_Available, max_len),
    "Ratings": pad_list(Ratings, max_len),
    "Route_link": pad_list(Route_links, max_len),
    "Route_name": pad_list(Route_names, max_len)
}

# Create DataFrame
df = pd.DataFrame(data)

In [40]:
# Save to CSV
df.to_csv('C:/Users/sinwa/projects/RED_BUS_project/Bus_details/Himachal_Buses.csv', index=False)



print("CSV file saved successfully.")

CSV file saved successfully.
